In [3]:
import pandas as pd
import numpy as np

# VARIABLES GLOBALES Y CONSTANTES
NIVEL_EMOCIONAL = ['Más positivo', 'Positivo', 'Neutro', 'Negativo', 'Más negativo']
NIVEL_APR_ENCST = ['Muy importante', 'Innovadora', 'Interesante', 'Poco importante']
NIVEL_SACIEDAD  = ['Totalmente', 'Normalmente', 'Medianamente', 'Poco', 'Nada']
RESTAURANTES    = ['Naturall', 'Tedy\'s Train', 'Restaurante cafetería Alcatraz', 'NDA']
TIPO_COMIDA     = ['Corriene', 'Rápida', 'Saludable', 'NDA']
RANGO_EDAD      = ['16 - 19', '20 - 23', '24 - 27']
SEMESTRE        = ['1', '2', '3', '4', '5', '6', '7', '8']
GENERO          = ['F', 'M']
COL_EM_ANTS = 15
COL_EM_DSPS = 16
COL_NVL_SCD = 10
COL_RESTRNT = 5

# CARGAR DATOS
cargar_excel = pd.read_excel('datos.xlsx')
df = pd.DataFrame(cargar_excel)
# CONVIERTE EL DATAFRAME EN UNA MATRIZ
TABLA_DATOS = df.to_numpy()


# DEVUELVE EL NUMERO DE PERSONAS QUE CAMBIARON DE LA EMOCION (A) A LA EMOCION (B)
def cambio_emocion(DATOS, emocion_A, emocion_B):
    cont = 0
    for i in range(0, len(DATOS)):
        if(DATOS[i][COL_EM_ANTS] == emocion_A and DATOS[i][COL_EM_DSPS] == emocion_B):
            cont = cont + 1
    return cont

# DEVUELVE LA LISTA CON LAS COMBINACIONES DEL CAMBIO DE LA EMOCION (A) A LA EMOCION (B)
def lista_cambio_emociones(DATOS, LST_EMOCIONES):
    emc_ants = []
    emc_desp = []
    for i in range(0, len(LST_EMOCIONES)):
        for j in range(0, len(LST_EMOCIONES)):
            if(cambio_emocion(DATOS, LST_EMOCIONES[i], LST_EMOCIONES[j]) != 0):
                emc_ants.append(LST_EMOCIONES[i])
                emc_desp.append(LST_EMOCIONES[j])
    return emc_ants, emc_desp

# DEVUELVE EL NUMERO DE PERSONAS EN CADA NIVEL DE SACIEDAD DEL CAMBIO DE LA EMOCION (A) A LA EMOCION (B)
def niveles_saciedad_cambio_emocion(DATOS, emocion_A, emocion_B, NVL_SACIEDAD):
    cont = np.full(len(NVL_SACIEDAD), 0)
    for i in range(0, len(DATOS)):
        if(DATOS[i][COL_EM_ANTS] == emocion_A and DATOS[i][COL_EM_DSPS] == emocion_B):
            for j in range(0, len(NVL_SACIEDAD)):
                if(DATOS[i][COL_NVL_SCD] == NIVEL_SACIEDAD[j]):
                    cont[j] = cont[j] + 1
    return cont

# DEVUELVE EL NUMERO DE PERSONAS QUE CAMBIAN DE LA EMOCION (A) A LA EMOCION (B) EN CADA RESTAURANTE
def cambio_emocion_restaurantes(DATOS, emocion_A, emocion_B, LST_RESTAURANTES):
    cont = np.full(len(LST_RESTAURANTES), 0)
    for i in range(0, len(DATOS)):
        if(DATOS[i][COL_EM_ANTS] == emocion_A and DATOS[i][COL_EM_DSPS] == emocion_B):
            for j in range(0, len(LST_RESTAURANTES)):
                if(DATOS[i][COL_RESTRNT] == LST_RESTAURANTES[j]):
                    cont[j] = cont[j] + 1
    return cont


# DEVUELVE EL NUMERO DE CADA CAMBIO DE LA EMOCION (A) A LA EMOCION (B) EN UN RESTAURANTE
# def restaurante_cambio_emociones(DATOS, LST_EMOCIONES, RESTAURANTE):

def funcion_principal():
    # COMBINACION DE LOS CAMBIO DE EMOCION 
    emc_ants, emc_desp = lista_cambio_emociones(TABLA_DATOS, NIVEL_EMOCIONAL)
    
    # SE CREA EL DOCUMENTO EN EL CUAL SE VAN A GUARDAR LOS DATOS
    documento = pd.ExcelWriter('ANALISIS_DATOS.xlsx', engine='xlsxwriter')

    # ==========================================================================
    arr_cam_emc = []
    num_cam_emc = []
    for i in range(0, len(emc_ants)):
        arr_cam_emc.append(emc_ants[i] + ' --> ' + emc_desp[i])
        num_cam_emc.append(cambio_emocion(TABLA_DATOS, emc_ants[i], emc_desp[i]))

    # DATOS QUE SE VAN A GUARDAR EN LA HOJA DE CALCULO DEFINIDA
    campo_hoja_01 = pd.DataFrame({'Cambio Emoción': arr_cam_emc, 'Cantidad Personas': num_cam_emc})

    # GUARDAR LOS DATOS EN LA HOJA DE CALCULO
    campo_hoja_01.to_excel(documento, sheet_name = 'CAMBIO DE EMOCIONES', index = False, startrow = 1, startcol = 1)


    for i in range(0, len(emc_ants)):
        n_s_c_e_tmp = niveles_saciedad_cambio_emocion(TABLA_DATOS, emc_ants[i], emc_desp[i], NIVEL_SACIEDAD)
        campo_tmp_01 = pd.DataFrame({'Cambio de': [], (emc_ants[i] + ' a ' + emc_desp[i]): []})
        campo_tmp_02 = pd.DataFrame({'Nivel Saciedad': NIVEL_SACIEDAD, 'Cantidad Personas': n_s_c_e_tmp})
        if(i == 0):
            campo_tmp_01.to_excel(documento, sheet_name = 'NIVEL SACIEDAD', index = False, startrow = i + 1, startcol = 1)
            campo_tmp_02.to_excel(documento, sheet_name = 'NIVEL SACIEDAD', index = False, startrow = i + 2, startcol = 1)
        else:
            campo_tmp_01.to_excel(documento, sheet_name = 'NIVEL SACIEDAD', index = False, startrow = (i * 9) + 1, startcol = 1)
            campo_tmp_02.to_excel(documento, sheet_name = 'NIVEL SACIEDAD', index = False, startrow = (i * 9) + 2, startcol = 1)


    for i in range(0, len(emc_ants)):
        c_e_r_tmp = cambio_emocion_restaurantes(TABLA_DATOS, emc_ants[i], emc_desp[i], RESTAURANTES)
        campo_tmp_01 = pd.DataFrame({'Cambio de': [], (emc_ants[i] + ' a ' + emc_desp[i]): []})
        campo_tmp_02 = pd.DataFrame({'Restaurante': RESTAURANTES, 'Cantidad Personas': c_e_r_tmp})
        if(i == 0):
            campo_tmp_01.to_excel(documento, sheet_name = 'RESTAURANTES', index = False, startrow = i + 1, startcol = 1)
            campo_tmp_02.to_excel(documento, sheet_name = 'RESTAURANTES', index = False, startrow = i + 2, startcol = 1)
        else:
            campo_tmp_01.to_excel(documento, sheet_name = 'RESTAURANTES', index = False, startrow = (i * 8) + 1, startcol = 1)
            campo_tmp_02.to_excel(documento, sheet_name = 'RESTAURANTES', index = False, startrow = (i * 8) + 2, startcol = 1)

    # ==========================================================================
    # GUARDA TODOS LOS CAMBIOS REALIZADOS SOBRE EL DOCUMENTO CREADO
    documento.save()
    display('TEMRINO')


# ==============================
funcion_principal()
# ==============================

# emc_ants, emc_desp = lista_cambio_emociones(TABLA_DATOS, NIVEL_EMOCIONAL)
# total = 0
# for i in range(0, len(emc_ants)):
#     tmp = cambio_emocion_restaurantes(TABLA_DATOS, emc_ants[i], emc_desp[i], RESTAURANTES)
#     display(emc_ants[i] + ' --> ' + emc_desp[i] + ' = ' + str(tmp))
#     total = total + tmp[4]

# display('TOTAL = ' + str(total))


'TEMRINO'